In [36]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
matches = pd.read_csv("../code/matches.csv")

In [8]:
matches['teamX_win'] = matches['SCORE_x'] > matches['SCORE_y']
matches['teamX_win'] = matches['teamX_win'].apply(lambda x : 1 if x else 0)

In [4]:
matches

YEAR_x       TEAM_x  SEED_x  SCORE_x  gameID  TEAMORIGINAL_x  \
0     2022      Gonzaga       1     93.0     137         Gonzaga   
1     2022     Boise St       8     53.0     138       Boise St.   
2     2022  Connecticut       5     63.0     139     Connecticut   
3     2022     Arkansas       4     75.0     140        Arkansas   
4     2022      Alabama       6     64.0     141         Alabama   
..     ...          ...     ...      ...     ...             ...   
57    2022      Houston       5     44.0     195         Houston   
58    2022       Kansas       1     76.0     196          Kansas   
59    2022         Duke       2     77.0     197            Duke   
60    2022    Villanova       2     65.0     198       Villanova   
61    2022    NCarolina       8     69.0     199  North Carolina   

   Closest Match_x afterWin_winLoss_Win_percent_x_x  afterWin_winLoss_MOV_x_x  \
0          gonzaga                            82.7%                      13.8   
1         boise st                            59.8%                       4.4   
2      connecticut                            73.0%                       9.5   
3         arkansas                            61.9%                       5.5   
4          alabama                            58.7%                       4.3   
..             ...                              ...                       ...   
57         houston                            65.8%                       6.7   
58          kansas                            79.6%                      11.4   
59            duke                            81.5%                      13.5   
60       villanova                            70.7%                       7.5   
61      n carolina                            73.2%                       9.5   

    afterWin_winLoss_ATS_x_x  ...  AST_y  STL_y  BLK_y  TOV_y   PF_y  Year_y  \
0                        1.0  ...  391.0  255.0  130.0  339.0  466.0  2022.0   
1                        0.6  ...  522.0  277.0  180.0  527.0  621.0  2022.0   
2                        1.0  ...  465.0  170.0  138.0  478.0  539.0  2022.0   
3                       -0.7  ...  515.0  173.0   84.0  326.0  475.0  2022.0   
4                        0.1  ...  491.0  182.0   65.0  365.0  446.0  2022.0   
..                       ...  ...    ...    ...    ...    ...    ...     ...   
57                       1.2  ...  451.0  229.0   83.0  376.0  566.0  2022.0   
58                       0.4  ...  523.0  328.0  118.0  352.0  550.0  2022.0   
59                       0.3  ...    NaN    NaN    NaN    NaN    NaN     NaN   
60                       0.9  ...  613.0  251.0  172.0  484.0  637.0  2022.0   
61                       0.3  ...  613.0  251.0  172.0  484.0  637.0  2022.0   

   seedDiff  isUnderdog_x  isUnderdog_y teamX_win  
0        15          True         False         1  
1         1          True         False         0  
2         7          True         False         0  
3         9          True         False         1  
4         5          True         False         0  
..      ...           ...           ...       ...  
57        3         False          True         0  
58        9          True         False         1  
59        6          True         False         0  
60        1         False          True         0  
61        7         False          True         0  

[62 rows x 109 columns]

In [9]:
matches = matches.drop(columns = ['YEAR_x', 
                        'TEAM_x', 
                        'SEED_x', 
                        'SCORE_x', 
                        'gameID', 
                        'TEAMORIGINAL_x', 
                        'Closest Match_x', 
                        'Team Original_x_x', 
                        'Team_x_x', 
                        'Team Original_y_x', 
                        'Team_y_x', 
                        'Year_x', 
                        'YEAR_y', 
                        'TEAM_y',
                        'SEED_y',
                        'SCORE_y',
                        'TEAMORIGINAL_y',
                        'Closest Match_y',
                        'Team Original_x_y',
                         'Team_x_y',
                         'Team Original_y_y',
                         'Team_y_y',
                         'TEAM_lower_y',
                        'TEAM_lower_x',
                        'Team_x',
                        'year_x',
                         'Team_y',
                         'year_y', 
                         'Year_y'])

matches['isUnderdog_x'] = matches['isUnderdog_x'].apply(lambda x : 1 if x else 0)
matches['isUnderdog_y'] = matches['isUnderdog_y'].apply(lambda x : 1 if x else 0)

In [33]:
def strip_and_convert(cell):
    cleaned_cell = str(cell).replace('%', '')  
    return float(cleaned_cell) if cleaned_cell else None
    
matches = matches.map(strip_and_convert)

matches['underdogPerc_interaction_percent_y'] = matches['asUnderdog_winLoss_Win_percent_x_y'] * matches['isUnderdog_y']
matches['underdogPerc_interaction_MOV_y'] = matches['asUnderdog_winLoss_MOV_x_y'] * matches['isUnderdog_y']
matches['underdogPerc_interaction_ATS_y'] = matches['asUnderdog_winLoss_ATS_x_y'] * matches['isUnderdog_y']

matches['underdogPerc_interaction_percent_x'] = matches['asUnderdog_winLoss_Win_percent_x_x'] * matches['isUnderdog_x']
matches['underdogPerc_interaction_MOV_x'] = matches['asUnderdog_winLoss_MOV_x_x'] * matches['isUnderdog_x']
matches['underdogPerc_interaction_ATS_x'] = matches['asUnderdog_winLoss_ATS_x_x'] * matches['isUnderdog_x']
matches['RSCI Top 100_x'] = matches['RSCI Top 100_x'].apply(lambda x : 0 if pd.isna(x) else x)
matches['RSCI Top 100_y'] = matches['RSCI Top 100_y'].apply(lambda x : 0 if pd.isna(x) else x)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
57    32.5
58     0.0
59     0.0
60    42.7
61    42.7
Name: underdogPerc_interaction_percent_y, Length: 62, dtype: float64

In [40]:
data = matches.dropna()

target = data["teamX_win"]
data = data.drop(columns=["teamX_win"])

train_data, test_data, train_target, test_target = train_test_split(
    data, target, test_size=0.25, random_state=999)

col_names = ['afterWin_winLoss_Win_percent_x_x',
 'afterWin_winLoss_MOV_x_x',
 'afterWin_winLoss_ATS_x_x',
 'underdogPerc_interaction_percent_x',
 'underdogPerc_interaction_MOV_x',
 'underdogPerc_interaction_ATS_x',
 'away_winLoss_MOV_x_x',
 'away_winLoss_ATS_x_x',
 'neutralSite_winLoss_Win_percent_x_x',
 'neutralSite_winLoss_MOV_x_x',
 'neutralSite_winLoss_ATS_x_x',
 'overallWinPercentage_Win_percent_x_x',
 'overallWinPercentage_MOV_x_x',
 'overallWinPercentage_ATS_x_x',
 'RSCI Top 100_x',
 'upperclassmen_x',
 'SRS_x',
 'SOS_x',
 'Tm_x',
 'Opp_x',
 'MP_x',
 'FG_x',
 'FGA_x',
 'FG%_x',
 '3P_x',
 '3PA_x',
 '3P%_x',
 'FT_x',
 'FTA_x',
 'FT%_x',
 'ORB_x',
 'TRB_x',
 'AST_x',
 'STL_x',
 'BLK_x',
 'TOV_x',
 'PF_x',
 'afterWin_winLoss_Win_percent_x_y',
 'afterWin_winLoss_MOV_x_y',
 'afterWin_winLoss_ATS_x_y',
 'underdogPerc_interaction_percent_y',
 'underdogPerc_interaction_MOV_y',
 'underdogPerc_interaction_ATS_y',
 'away_winLoss_Win_percent_x_y',
 'away_winLoss_MOV_x_y',
 'away_winLoss_ATS_x_y',
 'neutralSite_winLoss_Win_percent_x_y',
 'neutralSite_winLoss_MOV_x_y',
 'neutralSite_winLoss_ATS_x_y',
 'overallWinPercentage_Win_percent_x_y',
 'overallWinPercentage_MOV_x_y',
 'overallWinPercentage_ATS_x_y',
 'RSCI Top 100_y',
 'upperclassmen_y',
 'SRS_y',
 'SOS_y',
 'Tm_y',
 'Opp_y',
 'MP_y',
 'FG_y',
 'FGA_y',
 'FG%_y',
 '3P_y',
 '3PA_y',
 '3P%_y',
 'FT_y',
 'FTA_y',
 'FT%_y',
 'ORB_y',
 'TRB_y',
 'AST_y',
 'STL_y',
 'BLK_y',
 'TOV_y',
 'PF_y',
 'seedDiff',
 'isUnderdog_x',
 'isUnderdog_y']

numerical = train_data[col_names].values
numerical_test = test_data[col_names].values

model1 = LogisticRegression(max_iter=10000)
model1.fit(X=numerical, y=train_target)

predicted = model1.predict(numerical_test)
accuracy_model1 = accuracy_score(y_true=test_target, y_pred=predicted)
mse_1 = mean_squared_error(test_target, predicted)
r2_1 = r2_score(test_target, predicted)
print(mse_1)
print(r2_1)
print(accuracy_model1)

0.21428571428571427
-0.050000000000000044
0.7857142857142857


In [42]:
col_names_2 = [
 'RSCI Top 100_x',
 'upperclassmen_x',
 'SRS_x',
 'SOS_x',
 'Tm_x',
 'Opp_x',
 'MP_x',
 'FG_x',
 'FGA_x',
 'FG%_x',
 '3P_x',
 '3PA_x',
 '3P%_x',
 'FT_x',
 'FTA_x',
 'FT%_x',
 'ORB_x',
 'TRB_x',
 'AST_x',
 'STL_x',
 'BLK_x',
 'TOV_x',
 'PF_x',
 'RSCI Top 100_y',
 'upperclassmen_y',
 'SRS_y',
 'SOS_y',
 'Tm_y',
 'Opp_y',
 'MP_y',
 'FG_y',
 'FGA_y',
 'FG%_y',
 '3P_y',
 '3PA_y',
 '3P%_y',
 'FT_y',
 'FTA_y',
 'FT%_y',
 'ORB_y',
 'TRB_y',
 'AST_y',
 'STL_y',
 'BLK_y',
 'TOV_y',
 'PF_y',
 'seedDiff',
 'isUnderdog_x',
 'isUnderdog_y']

numerical_2 = train_data[col_names_2].values
numerical_test_2 = test_data[col_names_2].values

model2 = LogisticRegression(max_iter=10000)
model2.fit(X=numerical_2, y=train_target)

predicted_2 = model2.predict(numerical_test_2)
accuracy_model2 = accuracy_score(y_true=test_target, y_pred=predicted_2)
mse_2 = mean_squared_error(test_target, predicted_2)
r2_2 = r2_score(test_target, predicted_2)
print(mse_2)
print(r2_2)
print(accuracy_model2)

0.07142857142857142
0.65
0.9285714285714286


In [ ]:
matches['underdogPerc_interaction_percent_y'] = matches['asUnderdog_winLoss_Win_percent_x_y'] * matches['isUnderdog_y']
matches['underdogPerc_interaction_MOV_y'] = matches['asUnderdog_winLoss_MOV_x_y'] * matches['isUnderdog_y']
matches['underdogPerc_interaction_ATS_y'] = matches['asUnderdog_winLoss_ATS_x_y'] * matches['isUnderdog_y']

matches['underdogPerc_interaction_percent_x'] = matches['asUnderdog_winLoss_Win_percent_x_x'] * matches['isUnderdog_x']
matches['underdogPerc_interaction_MOV_x'] = matches['asUnderdog_winLoss_MOV_x_x'] * matches['isUnderdog_x']
matches['underdogPerc_interaction_ATS_x'] = matches['asUnderdog_winLoss_ATS_x_x'] * matches['isUnderdog_x']

col_names_2 = [
 'RSCI Top 100_x',
 'upperclassmen_x',
 'SRS_x',
 'SOS_x',
 'Tm_x',
 'Opp_x',
 'MP_x',
 'FG_x',
 'FGA_x',
 'FG%_x',
 '3P_x',
 '3PA_x',
 '3P%_x',
 'FT_x',
 'FTA_x',
 'FT%_x',
 'ORB_x',
 'TRB_x',
 'AST_x',
 'STL_x',
 'BLK_x',
 'TOV_x',
 'PF_x',
 'RSCI Top 100_y',
 'upperclassmen_y',
 'SRS_y',
 'SOS_y',
 'Tm_y',
 'Opp_y',
 'MP_y',
 'FG_y',
 'FGA_y',
 'FG%_y',
 '3P_y',
 '3PA_y',
 '3P%_y',
 'FT_y',
 'FTA_y',
 'FT%_y',
 'ORB_y',
 'TRB_y',
 'AST_y',
 'STL_y',
 'BLK_y',
 'TOV_y',
 'PF_y',
 'seedDiff',
 'isUnderdog_x',
 'isUnderdog_y']

numerical_2 = train_data[col_names_2].values
numerical_test_2 = test_data[col_names_2].values

model2 = LogisticRegression(max_iter=10000)
model2.fit(X=numerical_2, y=train_target)

predicted_2 = model2.predict(numerical_test_2)
accuracy_model2 = accuracy_score(y_true=test_target, y_pred=predicted_2)
mse_2 = mean_squared_error(test_target, predicted_2)
r2_2 = r2_score(test_target, predicted_2)
print(mse_2)
print(r2_2)
print(accuracy_model2)